In [1]:
%run ../ipynb_util_tars.py

In [2]:
%env WANDB_PROJECT=thesis

env: WANDB_PROJECT=thesis


In [3]:
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# LLaMA2 7B

## Load ZO_UP data

In [4]:
%run ../ipynb_load_data_natural.py

{'SDG': Value(dtype='int64', id=None), 'ABSTRACT': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'sdg_desc_short': Value(dtype='string', id=None), 'sdg_desc_long': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None)}
Example instance:	 {'SDG': 8, 'ABSTRACT': 'The scheme gives enterprises with business activity in Norway a tax credit on their R&D projects. The R&D content must be approved by the Research Council of Norway ex ante. In 2009, the cap on expenses per enterprise for intramural R&D projects increased to NOK 5.5 million (previously it was N0K 4 million), and NOK11 million (previously it was NOK 8 million) for projects conducted at an R&D institution.', 'id': None, 'sdg_desc_short': None, 'sdg_desc_long': None, '__index_level_0__': 492}
id2label: {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8', 8: '9', 9: '10', 10: '11', 11: '12', 12: '13', 13: '14', 14: '15', 15: '16', 16: '17'}
label2id: {'1': 0, '2': 1, '

In [5]:
import pprint
pprint.pprint(id2label)

{0: '1',
 1: '2',
 2: '3',
 3: '4',
 4: '5',
 5: '6',
 6: '7',
 7: '8',
 8: '9',
 9: '10',
 10: '11',
 11: '12',
 12: '13',
 13: '14',
 14: '15',
 15: '16',
 16: '17'}


## Model

In [6]:
from transformers.data import DataCollatorWithPadding
from transformers.models.llama import LlamaForSequenceClassification
from peft import LoraConfig, TaskType, get_peft_model
from transformers import BitsAndBytesConfig

MODEL_NAME = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,
# )

model = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    id2label=id2label,
    label2id=label2id,
    token=HF_TOKEN,
    #quantization_config=bnb_config
).bfloat16()
model.config.pad_token_id = tokenizer.pad_token_id
# model.resize_token_embeddings(len(tokenizer))

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

encoded_dataset = dataset.map(
    preprocess_data(tokenizer), batched=True, remove_columns=dataset["train"].column_names
)
encoded_dataset.set_format("torch")

data_collator = DataCollatorWithPadding(tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 8,458,240 || all params: 6,615,871,488 || trainable%: 0.1278


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <enum 'DatasetType'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <enum 'DatasetType'>: __main__.DatasetType has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Map:   0%|          | 0/630 [00:00<?, ? examples/s]

Map:   0%|          | 0/271 [00:00<?, ? examples/s]

In [7]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer, EvalPrediction

BATCH_SIZE = 4
METRIC_NAME = "accuracy"

args = TrainingArguments(
    f"{CHECKPOINT_PATH}/final/{MODEL_NAME}-noq",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    #eval_steps=50,
    #save_strategy="steps",
    #save_steps=50,
    #logging_steps=50,
    learning_rate=1e-4,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=5,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=METRIC_NAME,
    seed=SEED,
    report_to="wandb",
)

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dvdblk (ngmi). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.911400,1.587696,0.501845,0.556003,0.501845,0.479568
2,0.758500,1.043169,0.723247,0.756191,0.723247,0.716954
3,0.228100,0.986561,0.778598,0.797714,0.778598,0.781883
4,0.033200,0.934318,0.785978,0.796855,0.785978,0.788406
5,0.008700,0.934128,0.782288,0.791624,0.782288,0.783847


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user

TrainOutput(global_step=790, training_loss=0.7879701477062853, metrics={'train_runtime': 1211.7668, 'train_samples_per_second': 2.6, 'train_steps_per_second': 0.652, 'total_flos': 6.27521076854784e+16, 'train_loss': 0.7879701477062853, 'epoch': 5.0})

In [9]:
trainer.evaluate()

{'eval_loss': 0.9343181848526001,
 'eval_accuracy': 0.7859778597785978,
 'eval_precision': 0.796855181406065,
 'eval_recall': 0.7859778597785978,
 'eval_f1': 0.7884059640891449,
 'eval_runtime': 36.7269,
 'eval_samples_per_second': 7.379,
 'eval_steps_per_second': 1.852,
 'epoch': 5.0}

In [10]:
from sklearn.metrics import classification_report

model.eval()

# manual evaluation to show classifcation_report
true_labels = []
logits = []

for batch in encoded_dataset["test"]:
    batch = {k: v.to(trainer.args.device).unsqueeze(0) for k, v in batch.items()}
    label = batch.pop("label")

    # Forward pass
    with torch.no_grad():
        out = model(**batch)

    true_labels.append(label.item())
    logits.extend(out.logits.tolist())

probabilites = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
pred_labels = torch.argmax(probabilites, dim=-1).tolist()

report = classification_report(true_labels, pred_labels, target_names=[f"SDG {id2label[i]}" for i in range(len(labels))])

In [11]:
print(report)

              precision    recall  f1-score   support

       SDG 1       0.88      0.82      0.85        17
       SDG 2       0.89      0.94      0.91        17
       SDG 3       0.93      0.82      0.88        17
       SDG 4       1.00      0.94      0.97        17
       SDG 5       0.88      0.82      0.85        17
       SDG 6       0.83      0.94      0.88        16
       SDG 7       0.92      0.75      0.83        16
       SDG 8       0.52      0.76      0.62        17
       SDG 9       0.68      0.76      0.72        17
      SDG 10       0.41      0.41      0.41        17
      SDG 11       0.75      0.71      0.73        17
      SDG 12       0.67      0.59      0.62        17
      SDG 13       0.88      0.88      0.88        17
      SDG 14       1.00      1.00      1.00        17
      SDG 15       0.94      0.88      0.91        17
      SDG 16       0.62      0.59      0.61        17
      SDG 17       0.00      0.00      0.00         1

    accuracy              